In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
CLIENT_ID = 'Your ID' 
CLIENT_SECRET = 'Your Secret' 
VERSION = '20200101' 
radius = 5
LIMIT = 5

In [4]:
def drop_repeated_postcode(data):
    temp_list = []
    for i in range(1,data.shape[0]):
        if data.iloc[i,1] == data.iloc[i-1,1]:
            temp_list.append(i)
    return data.drop(temp_list).reset_index(drop = True)

def make_address(column, added_city):
    for i in range(column.shape[0]):
        column.iloc[i,0] = column.iloc[i,0] + ',' + added_city
    return column

def get_coordinate(address):
    address = address
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    return latitude, longitude

def add_columns_for_coordiate(data):
    data['Latitude'] = None
    data['Longitude'] = None
    
def get_listof_cooridate(data):
    latitude_list = []
    longitude_list = []
    for i in data['District']:
        latitude, longitude = get_coordinate(i)
        latitude_list.append(latitude)
        longitude_list.append(longitude)
        time.sleep(90)
    data['Latitude'] = latitude_list
    data['Longitude'] = longitude_list
    return data

#Function to find near venues for every data point
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

def plot_district_graph(address,data,zoom):
    address = address
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

    map_to_plot = folium.Map(location=[latitude, longitude], zoom_start=zoom)

    for lat, lng, neighbourhood in zip(data['Neighborhood Latitude'], 
                                       data['Neighborhood Longitude'], 
                                       data['Neighborhood']):
        label = '{}'.format(neighbourhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7,
                parse_html=False).add_to(map_to_plot)  
    
    return map_to_plot

def plot_cluster_map(address,zoom):
    address = address
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    map_clusters = folium.Map([latitude, longitude], zoom_start=10)

    # set color scheme for the clusters
    x = np.arange(k)
    ys = [i + x + (i*x)**2 for i in range(k)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(copy_all_data['Neighborhood Latitude'], copy_all_data['Neighborhood Longitude'], copy_all_data['Neighborhood'], copy_all_data['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
    return map_clusters

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [5]:
HK = pd.read_csv('HK.csv')
SG = pd.read_csv('SG.csv')
shanghai = pd.read_csv('shanghai.csv')
beijing = pd.read_csv('beijing.csv')
TP = pd.read_csv('TP.csv')

In [6]:
HK_Venues = getNearbyVenues(HK['District'], HK['Latitude'], HK['Longitude'])
SG_Venues = getNearbyVenues(SG['District'], SG['Latitude'], SG['Longitude'])
shanghai_Venues = getNearbyVenues(shanghai['District'], shanghai['Latitude'], shanghai['Longitude'])
beijing_Venues = getNearbyVenues(beijing['District'], beijing['Latitude'], beijing['Longitude'])
TP_Venues = getNearbyVenues(TP['District'], TP['Latitude'], TP['Longitude'])

Central and Western,Hong Kong
Wan Chai,Hong Kong
Sham Shui Po,Hong Kong
Kowloon City,Hong Kong
Kwun Tong,Hong Kong
Wong Tai Sin,Hong Kong
Yau Tsim Mong,Hong Kong
Kwai Tsing,Hong Kong
Sai Kung,Hong Kong
Sha Tin,Hong Kong
Tai Po,Hong Kong
Tsuen Wan,Hong Kong
Tuen Mun,Hong Kong
Yuen Long,Hong Kong
Raffles Place,Singapore
Anson,Singapore
Bukit Merah,Singapore
Telok Blangah,Singapore
Pasir Panjang,Singapore
High Street,Singapore
Middle Road,Singapore
Little India,Singapore
Orchard,Singapore
Ardmore,Singapore
Watten Estate,Singapore
Balestier,Singapore
Macpherson,Singapore
Geylang,Singapore
Katong,Singapore
Bedok,Singapore
Loyang,Singapore
Simei,Singapore
Serangoon Garden,Singapore
Bishan,Singapore
Upper Bukit Timah,Singapore
Jurong,Singapore
Hillview,Singapore
Lim Chu Kang,Singapore
Kranji,Singapore
Upper Thomson,Singapore
Yishun,Singapore
Seletar,Singapore
Jinshan District ,Shanghai
Songjiang District ,Shanghai
Qingpu District ,Shanghai
Jiading District ,Shanghai
Baoshan District ,Shanghai

In [7]:
data_list = [HK_Venues, SG_Venues, shanghai_Venues, beijing_Venues, TP_Venues]

In [8]:
onehot_list = []
grouped_list = []
for data in data_list:
    onehot = pd.get_dummies(data[['Venue Category']], prefix="", prefix_sep="")
    onehot['Neighborhood'] = data['Neighborhood']
    onehot_list.append(onehot)
    grouped = onehot.groupby('Neighborhood').mean().reset_index()
    grouped_list.append(grouped)    

In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [10]:
neighbor_venues_sorted_list = []
for grouped in grouped_list:
    num_top_venues = 5
    indicators = ['st', 'nd', 'rd']

    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))


    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

    for ind in np.arange(grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

    neighbor_venues_sorted_list.append(neighborhoods_venues_sorted)

In [11]:
data_for_clustering = pd.concat([grouped_list[0], grouped_list[1], grouped_list[2], grouped_list[3], grouped_list[4]], 
                                ignore_index=True).fillna(0)

all_data = pd.concat(data_list, ignore_index = True)

all_sorted_venues = pd.concat([neighbor_venues_sorted_list[0], neighbor_venues_sorted_list[1],
                               neighbor_venues_sorted_list[2], neighbor_venues_sorted_list[3],
                               neighbor_venues_sorted_list[4]], ignore_index = True)

/Users/cloudli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [12]:
k = 4

data_for_clustering = data_for_clustering.drop('Neighborhood',1)

kmeans = KMeans(n_clusters = k).fit(data_for_clustering)

kmeans.labels_

array([1, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 2, 1, 0, 2,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 1, 0],
      dtype=int32)

In [13]:
# add clustering labels
all_sorted_venues.insert(0, 'Cluster Labels', kmeans.labels_)

copy_all_data = all_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
copy_all_data = copy_all_data.join(all_sorted_venues.set_index('Neighborhood'), on='Neighborhood')

copy_all_data.head() 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Central and Western,Hong Kong",22.285039,114.158382,Apple ifc mall,22.284602,114.159072,Electronics Store,1,Hotel,Electronics Store,Spa,Snack Place,Shopping Mall
1,"Central and Western,Hong Kong",22.285039,114.158382,Four Seasons Hotel Hong Kong (香港四季酒店),22.286554,114.156929,Hotel,1,Hotel,Electronics Store,Spa,Snack Place,Shopping Mall
2,"Central and Western,Hong Kong",22.285039,114.158382,ifc mall (國際金融中心商場),22.284613,114.158156,Shopping Mall,1,Hotel,Electronics Store,Spa,Snack Place,Shopping Mall
3,"Central and Western,Hong Kong",22.285039,114.158382,Pool Terrace (池畔餐廳),22.286218,114.157583,Snack Place,1,Hotel,Electronics Store,Spa,Snack Place,Shopping Mall
4,"Central and Western,Hong Kong",22.285039,114.158382,The Spa at Four Seasons,22.286279,114.157623,Spa,1,Hotel,Electronics Store,Spa,Snack Place,Shopping Mall


In [14]:
copy_all_data.head(1)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Central and Western,Hong Kong",22.285039,114.158382,Apple ifc mall,22.284602,114.159072,Electronics Store,1,Hotel,Electronics Store,Spa,Snack Place,Shopping Mall


In [15]:
plot_cluster_map("Hong Kong", 11.5)

/Users/cloudli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [119]:
plot_cluster_map("Singapore", 12)

/Users/cloudli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
plot_cluster_map("Taipei", 13)

/Users/cloudli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [120]:
plot_cluster_map("Shanghai", 11.5)

/Users/cloudli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [124]:
plot_cluster_map("Beijing", 1)

/Users/cloudli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
all_sorted_venues.iloc[:14]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,"Central and Western,Hong Kong",Hotel,Electronics Store,Spa,Snack Place,Shopping Mall
1,0,"Kowloon City,Hong Kong",Dessert Shop,Market,Halal Restaurant,Park,Hotpot Restaurant
2,0,"Kwai Tsing,Hong Kong",Tunnel,Café,Chinese Restaurant,French Restaurant,Dumpling Restaurant
3,2,"Kwun Tong,Hong Kong",Burger Joint,Gym,Chinese Restaurant,Coffee Shop,Electronics Store
4,0,"Sai Kung,Hong Kong",Bar,Café,Market,Chinese Restaurant,Park
5,0,"Sha Tin,Hong Kong",Department Store,Café,Supermarket,Ice Cream Shop,Dumpling Restaurant
6,0,"Sham Shui Po,Hong Kong",Italian Restaurant,Malay Restaurant,Hong Kong Restaurant,Cha Chaan Teng,Snack Place
7,2,"Tai Po,Hong Kong",Noodle House,Chinese Restaurant,Coffee Shop,Department Store,Pub
8,0,"Tsuen Wan,Hong Kong",Sushi Restaurant,Cantonese Restaurant,Cha Chaan Teng,Noodle House,Dessert Shop
9,2,"Tuen Mun,Hong Kong",Zoo,Cantonese Restaurant,Steakhouse,Park,Coffee Shop


In [17]:
all_sorted_venues.iloc[14:42]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,2,"Anson,Singapore",Gym / Fitness Center,Hotel,Bakery,Salad Place,Coffee Shop
15,1,"Ardmore,Singapore",Hotel,Café,Hotel Bar,Sushi Restaurant,Convenience Store
16,0,"Balestier,Singapore",Chinese Restaurant,Hotel,Indian Restaurant,Park,Convenience Store
17,2,"Bedok,Singapore",French Restaurant,Bakery,Coffee Shop,Breakfast Spot,Chinese Restaurant
18,2,"Bishan,Singapore",Japanese Restaurant,Shaanxi Restaurant,Coffee Shop,Chinese Restaurant,Food Court
19,1,"Bukit Merah,Singapore",Bookstore,Yoga Studio,Cupcake Shop,Ice Cream Shop,Churrascaria
20,0,"Geylang,Singapore",Chinese Restaurant,Vegetarian / Vegan Restaurant,Basketball Court,Dim Sum Restaurant,Cantonese Restaurant
21,2,"High Street,Singapore",Art Gallery,French Restaurant,Monument / Landmark,Coffee Shop,Shopping Mall
22,0,"Hillview,Singapore",Bistro,Beer Bar,Italian Restaurant,Beer Garden,Non-Profit
23,0,"Jurong,Singapore",Farmers Market,Zoo Exhibit,Cupcake Shop,Chocolate Shop,Churrascaria


In [18]:
all_sorted_venues.iloc[42:49]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
42,3,"Baoshan District ,Shanghai",Metro Station,Zhejiang Restaurant,Fast Food Restaurant,Bubble Tea Shop,Buddhist Temple
43,0,"Minhang District ,Shanghai",Park,Metro Station,Malay Restaurant,Chinese Restaurant,Zhejiang Restaurant
44,1,"Nanhui District ,Shanghai",Bakery,Fast Food Restaurant,Burger Joint,Hotel,Theater
45,1,"Pudong New Area ,Shanghai",Zhejiang Restaurant,Sports Bar,Hotel,Grocery Store,Coffee Shop
46,0,"Qingpu District ,Shanghai",Bubble Tea Shop,Pharmacy,Zhejiang Restaurant,Fast Food Restaurant,Buddhist Temple
47,0,"Shanghai City ,Shanghai",Garden,Buddhist Temple,Clothing Store,Historic Site,Dumpling Restaurant
48,1,"Songjiang District ,Shanghai",Hotel,Chinese Restaurant,Zhejiang Restaurant,Fast Food Restaurant,Bubble Tea Shop


In [19]:
all_sorted_venues.iloc[49:51]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
49,0,"Beijing City ,Beijing",Historic Site,Plaza,Park,Concert Hall,Theme Park
50,0,"Miyun County ,Beijing",Theme Park,Fast Food Restaurant,Plaza,Park,Historic Site


In [20]:
all_sorted_venues.iloc[51:]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
51,1,"Beitou District,Taipei",Farmers Market,Tea Room,Italian Restaurant,Hotel,Dessert Shop
52,0,"Daan District,Taipei",Café,Tea Room,Chinese Restaurant,Park,Train Station
53,2,"Datong District,Taipei",Taiwanese Restaurant,Chinese Restaurant,Dessert Shop,Gym / Fitness Center,Train Station
54,1,"Nangang District,Taipei",Train Station,Hotel,Breakfast Spot,Coffee Shop,Movie Theater
55,2,"Neihu District,Taipei",Chinese Restaurant,Coffee Shop,Convenience Store,Gym,Train Station
56,1,"Shilin District,Taipei",Bookstore,Ice Cream Shop,Soup Place,Shoe Store,Farmers Market
57,1,"Songshan District,Taipei",Train Station,Night Market,Bookstore,Food Stand,Seafood Restaurant
58,2,"Wanhua District,Taipei",Bakery,Snack Place,Dessert Shop,Coffee Shop,Taiwanese Restaurant
59,2,"Wenshan District,Taipei",Coffee Shop,Farmers Market,Ice Cream Shop,Japanese Restaurant,Train Station
60,1,"Xinyi District,Taipei",Hunan Restaurant,Hotel,Japanese Restaurant,Plaza,Bookstore
